<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

In [33]:
class CONFIG:
  batch_size = 16
  device = 'tpu'
  epochs = 1
  input_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Preprocessed'
  mode = 'dev'
  model = 'microsoft/deberta-base'
  output_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output'
  sequence_max_length = 466

## Packages

In [34]:
!pip install transformers
!pip install tokenizers

!pip install --force-reinstall git+https://github.com/xiaoyufan/nbme.git

!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Cloning https://github.com/xiaoyufan/nbme.git to /tmp/pip-req-build-a3cerrmu
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-a3cerrmu
  Created wheel for nbme: filename=nbme-1.0-py3-none-any.whl size=2308 sha256=20e816c8cee236dbac693f471203d98c9a38e89b004ebda0ff94f7b83235545d
  Stored in directory: /tmp/pip-ephem-wheel-cache-s8qyjj9w/wheels/e2/dc/dd/b061e30220e414c0a096e4fbaddd405c1b526d1453d9a447e3
Successfully built nbme
  Attempting uninstall: nbme
    Found existing installation: nbme 1.0
    Uninstalling nbme-1.0:
      Successfully uninstalled nbme-1.0


  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)


In [35]:
import pandas as pd
import torch
import torch.nn as nn

from nbme_utils import scoring
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset

## Device

In [36]:
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if CONFIG.device == 'tpu':
  assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

device

device(type='xla', index=1)

## Tokenizer

In [37]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG.model)
tokenizer.save_pretrained(f'{CONFIG.output_dir}/tokenizer')

('/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/vocab.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/merges.txt',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer.json')

## Data Loading

### Defining Dataset

In [38]:
class NBMEDataset(Dataset):
  def __init__(self, data, tokenizer, config):
    self.data = data
    self.tokenizer = tokenizer
    self.config = config

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sample = self.data.iloc[idx]

    x = self.tokenizer(
      sample["pn_history"],
      sample["feature_text"],
      # TODO: Compute max length of sequences
      max_length=self.config['sequence_max_length'],
      padding='max_length',
      return_offsets_mapping=True,
    )

    return x

### Dataset Loader

In [39]:
train = pd.read_csv(f'{CONFIG.input_dir}/train.csv')
valid = pd.read_csv(f'{CONFIG.input_dir}/validate.csv')
train.shape, valid.shape

((11342, 8), (2958, 8))

In [40]:
if CONFIG.mode == 'dev':
  train = train.sample(n=800, random_state=0).reset_index(drop=True)
  valid = valid.sample(n=200, random_state=0).reset_index(drop=True)
train.shape, valid.shape

((800, 8), (200, 8))

In [41]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text
0,00225_008,0,225,8,"['drinks coffee', 'drinks energy drinks']","['461 467;483 489', '461 467;506 519']",17 y/o previously healthy male here with heart...,Caffeine-use
1,81979_803,8,81979,803,[],[],67yo woman presents with 3 weeks of difficulty...,Auditory-hallucination-once
2,31800_311,3,31800,311,[],[],HPI: Mr. Hamilton is a 35 year old male who pr...,No-blood-in-stool
3,20158_216,2,20158,216,['44'],['22 24'],"Ms Dolores Montgmery, 44 yo F c/o Irregular me...",44-year
4,21591_201,2,21591,201,[],[],Dolores Montgomery is a 44 year old female wit...,Last-Pap-smear-I-year-ago


In [42]:
train_dataset = NBMEDataset(train, tokenizer, CONFIG)
valid_dataset = NBMEDataset(valid, tokenizer, CONFIG)

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.batch_size,
    shuffle=True,)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=CONFIG.batch_size,
    shuffle=False,)

## Model

In [43]:
class NBMEDebertaBaseline(nn.Module):
  def __init__(self):
    super().__init__()

    config = AutoConfig.from_pretrained(CONFIG.model, output_hidden_states=True)
    self.model = AutoModel.from_pretrained(CONFIG.model, config=config)

    self.fc = nn.Linear(config.hidden_size, 1)

  def forward(self, data):
    last_out = self.model(**data)[0]
    logits = self.fc(last_out)
    return logits

In [44]:
model = NBMEDebertaBaseline()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Training

## Inference